In [2]:
import pandas as pd
df = pd.read_json("data.json")

In [2]:
df

,text,label
0,"Date: Wed, 20 Nov 1996 23:30:42 GMT\nServer: N...",course
1,"Date: Tue, 26 Nov 1996 18:47:26 GMT\nServer: N...",course
2,"Date: Mon, 11 Nov 1996 16:02:43 GMT\nServer: N...",course
3,MIME-Version: 1.0\nServer: CERN/3.0\nDate: Sun...,course
4,"Date: Tue, 26 Nov 1996 19:11:28 GMT\nServer: A...",course
...,...,...
6618,"Date: Tue, 26 Nov 1996 03:58:29 GMT\nServer: A...",student
6619,MIME-Version: 1.0\nServer: CERN/3.0\nDate: Thu...,student
6620,"Date: Thu, 21 Nov 1996 23:05:17 GMT\nServer: N...",student
6621,"Date: Tue, 26 Nov 1996 03:46:09 GMT\nServer: N...",student


In [5]:
df.size

13246

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'doc2query/all-with_prefix-t5-base-v1'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
prefix = "answer2question"
outputColumn = []
cont = 0
for i in df['text']:
    text = prefix+": "+ i
    input_ids = tokenizer.encode(text, max_length=384, truncation=True, return_tensors='pt')
    outputs = model.generate(
        input_ids=input_ids,
        max_length=64,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=20)

    outputText = i

    for i in range(len(outputs)):
        query = tokenizer.decode(outputs[i], skip_special_tokens=True)
        outputText = outputText + ' ' +  query
    outputColumn.append(outputText)
    if cont % 100 == 0:
        print(cont)
        print(cont//132)
    cont += 1
df['expandedText'] = outputColumn

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0
0
100
0
200
0
300
0
400
0


KeyboardInterrupt: 

In [ ]:
df.to_json('expanded_data.json')

In [3]:
for i in df['text']:
    print(i)
    break

Date: Wed, 20 Nov 1996 23:30:42 GMT
Server: NCSA/1.5.2
Last-modified: Wed, 16 Oct 1996 04:38:33 GMT
Content-type: text/html
Content-length: 1361

571 Main Page


Computer Science 571
CONCEPTS IN PROGRAMMING LANGUAGES


INSTRUCTOR
Ellis Horowitz
Computer Science Dept.
University of Southern California
Los Angeles, Ca. 90089
OFFICE: 300 SAL, PHONE: 213-740-8056
EMAIL: horowitz@pollux.usc.edu
Office hrs:Mon-Wed: 10:30-Noon, Tues: 1:00-3:00pm


 Textbooks
 Course Overview
 Week-by-Week Topics
 Introduction
 Survey Form
 NTU Syllabus
 Programming Language References

Classnotes
571 Classnotes
Lex&Yacc Notes
Languages Directory
Sample Tests
README.571
New JavaScript Classnotes
New Java Classnotes

